## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Corner Brook,CA,48.9667,-57.9484,60.93,93,89,4.70,overcast clouds
1,1,Namatanai,PG,-3.6667,152.4333,80.73,81,95,6.35,moderate rain
2,2,Kununurra,AU,-15.7667,128.7333,98.58,20,96,6.91,overcast clouds
3,3,Chengdu,CN,30.6667,104.0667,75.09,73,40,11.18,scattered clouds
4,4,New Norfolk,AU,-42.7826,147.0587,55.76,56,14,1.99,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the coldest temperature you would like for your trip? "))
max_temp = float(input("What is the hottest temperature you would like for your trip? "))

What is the coldest temperature you would like for your trip? 75
What is the hottest temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Namatanai,PG,-3.6667,152.4333,80.73,81,95,6.35,moderate rain
3,3,Chengdu,CN,30.6667,104.0667,75.09,73,40,11.18,scattered clouds
8,8,Coahuayana,MX,18.7333,-103.6833,77.81,89,100,2.42,overcast clouds
10,10,Bathsheba,BB,13.2167,-59.5167,77.61,88,40,4.61,scattered clouds
14,14,Cabo Rojo,PR,18.0866,-67.1457,79.95,91,3,1.01,clear sky
19,19,Kieta,PG,-6.2167,155.6333,80.08,84,100,4.76,light rain
21,21,Alice Springs,AU,-23.7000,133.8833,87.35,8,0,9.22,clear sky
27,27,Esperance,AU,-33.8667,121.9000,75.04,73,63,6.53,broken clouds
28,28,Itoman,JP,26.1247,127.6694,87.82,70,40,13.80,scattered clouds
30,30,Tanout,NE,14.9709,8.8879,89.10,13,30,9.17,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
# Check for null values
preferred_cities_df.count()

City_ID                226
City                   226
Country                225
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Get the columns and the rows that are not null.
preferred_cities_df.isnull().sum()

clean_cities_df = preferred_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Namatanai,PG,-3.6667,152.4333,80.73,81,95,6.35,moderate rain
3,3,Chengdu,CN,30.6667,104.0667,75.09,73,40,11.18,scattered clouds
8,8,Coahuayana,MX,18.7333,-103.6833,77.81,89,100,2.42,overcast clouds
10,10,Bathsheba,BB,13.2167,-59.5167,77.61,88,40,4.61,scattered clouds
14,14,Cabo Rojo,PR,18.0866,-67.1457,79.95,91,3,1.01,clear sky
...,...,...,...,...,...,...,...,...,...,...
654,654,Sortino,IT,37.1566,15.0260,76.37,65,14,3.06,few clouds
655,655,Robstown,US,27.7903,-97.6689,76.75,82,1,0.00,clear sky
656,656,Nsanje,MW,-16.9200,35.2620,91.00,32,0,4.36,clear sky
670,670,Atar,MR,20.5169,-13.0499,91.31,24,22,3.33,few clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Namatanai,PG,80.73,moderate rain,-3.6667,152.4333,
3,Chengdu,CN,75.09,scattered clouds,30.6667,104.0667,
8,Coahuayana,MX,77.81,overcast clouds,18.7333,-103.6833,
10,Bathsheba,BB,77.61,scattered clouds,13.2167,-59.5167,
14,Cabo Rojo,PR,79.95,clear sky,18.0866,-67.1457,
19,Kieta,PG,80.08,light rain,-6.2167,155.6333,
21,Alice Springs,AU,87.35,clear sky,-23.7000,133.8833,
27,Esperance,AU,75.04,broken clouds,-33.8667,121.9000,
28,Itoman,JP,87.82,scattered clouds,26.1247,127.6694,
30,Tanout,NE,89.10,scattered clouds,14.9709,8.8879,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

In [11]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   225
Country                225
Max Temp               225
Current Description    225
Lat                    225
Lng                    225
Hotel Name             225
dtype: int64

In [13]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Namatanai,PG,80.73,moderate rain,-3.6667,152.4333,Seagulls Inn Namatani
3,Chengdu,CN,75.09,scattered clouds,30.6667,104.0667,Crowne Plaza Chengdu City Center
8,Coahuayana,MX,77.81,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
10,Bathsheba,BB,77.61,scattered clouds,13.2167,-59.5167,Atlantis Hotel
14,Cabo Rojo,PR,79.95,clear sky,18.0866,-67.1457,Hotel Perichi's
...,...,...,...,...,...,...,...
654,Sortino,IT,76.37,few clouds,37.1566,15.0260,casa nonna Gisaria
655,Robstown,US,76.75,clear sky,27.7903,-97.6689,Days Inn by Wyndham Robstown
656,Nsanje,MW,91.00,clear sky,-16.9200,35.2620,Nsanje Discovery Lodge
670,Atar,MR,91.31,few clouds,20.5169,-13.0499,Oumou Elghoura


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))